# The sensitivity analysis in pyPowsybl 

This notebook illustrates how to run a sensitivity analysis.


In [ ]:
pip install pypowsybl

In [ ]:
import pypowsybl as pp

In [ ]:
n6 = pp.network.create_metrix_tutorial_six_buses_network()

## 1 - DC Sensitivity analysis

To perform a sensitivity analysis, you must first define the “factors” you want to compute. What we call a factor is the impact of a small variation of a variable, typically the active power injection of a generator, a load or a phase shifter, on a function, typically the active power flow on a branch. 

To make the definition of those factors easier, `pypowsybl` provides a method to define the variables (injection, phase shifter) through their ids (variables_ids) and the functions through the branch ids (branches_ids). We obtain a matrix of sensitivities as a result:

In [ ]:
analysis = pp.sensitivity.create_dc_analysis()
analysis.add_branch_flow_factor_matrix(branches_ids=['S_SO_1', 'S_SO_2'], variables_ids=['SE_G'])
result = analysis.run(n6)
result.get_reference_matrix()

In [ ]:
result.get_sensitivity_matrix()

The result can be interpreted in the following way: an increase of 1 MW on generator SE_G impacts the lines S_SO_1 and S_SO_2 with a 0.3 MW increase of the active power flow from side 2 to side 1.

## 2 - Zone to zone sensitivity: PTDF computation

This zone to zone sensitivity feature is better known as Power Transfer Distribution Factor (PTDF).

Consider now that the node `SE` is now in Italy:

In [ ]:
n6b = pp.network.create_metrix_tutorial_six_buses_network()
n6b.update_substations(id=['SE'], TSO=['Terna'], country=['IT'])
n6b.update_substations(id=['NO', 'S', 'SO', 'N'], TSO=['RTE', 'RTE', 'RTE', 'RTE'], country=['FR', 'FR', 'FR', 'FR'])

In [ ]:
pp.loadflow.run_dc(n6b)

In [ ]:
n6b.get_substations()

In [ ]:
n6b.get_generators()

In [ ]:
n6b.get_loads()

In [ ]:
zone_fr = pp.sensitivity.create_country_zone(n6b, 'FR')
zone_it = pp.sensitivity.create_country_zone(n6b, 'IT')

In [ ]:
zone_fr.shift_keys_by_injections_ids

In [ ]:
params = pp.loadflow.Parameters(distributed_slack=False)
sa = pp.sensitivity.create_dc_analysis()
sa.set_zones([zone_fr, zone_it])

In [ ]:
sa.add_branch_flow_factor_matrix(branches_ids=['S_SE_1', 'S_SE_2'], variables_ids=['FR', 'IT'])

In [ ]:
ptdf_results = sa.run(n6b, params)
m1 = ptdf_results.get_branch_flows_sensitivity_matrix()

In [ ]:
m1

1 MW active power transfer from FR zone to IT zone will be responsible of a variation of 0.3 MW on the border line S_SE_1.

Let’s obtain that directly. After a sensitivity analysis where we should set the zones, we are able to ask for a FR zone to slack sensitivity, a FR to IT zone to zone sensitivity, a IT to FR zone to zone sensitivity and a IT zone to slack sensitivity, on all the border lines ‘S_SE_1’, ‘S_SE_2’, ‘SE_NE_1’ and ‘SE_NE_2’.

In [ ]:
sa = pp.sensitivity.create_dc_analysis()
sa.set_zones([zone_fr, zone_it])
sa.add_branch_flow_factor_matrix(branches_ids=['S_SE_1', 'S_SE_2', 'SE_NE_1', 'SE_NE_2'], variables_ids=['FR', ('FR', 'IT'), ('IT', 'FR'), 'IT'])
ptdf_result = sa.run(n6b, params)

In [ ]:
m2 = ptdf_result.get_branch_flows_sensitivity_matrix()
m2

We can see that:
- an increase of 1 MW on the FR zone net position leads to an increase of 0.2 MW on the flow of all the border lines
- moving the flow of 1 MW from FR to IT leads to an increase of 0.3 MW on the flow of lines S_SE_1 and S_SE_2 and a decrease of 0.2 MW on the flow of lines SE_NE_1 and SE_NE_2
- moving the flow of 1 MW from IT to FR leads to a decrease of 0.3 MW on the flow of lines S_SE_1 and S_SE_2 and an increase of 0.2 MW on the flow of lines SE_NE_1 and SE_NE_2
- an increase of 1 MW on the IT zone net position leads to a decrease of 0.1 MW on the flow of lines S_SE_1 and S_SE_2 and an increase of 0.4 MW on the flow of lines SE_NE_1 and SE_NE_2